In [1]:
import sys
from nbfinder import NotebookFinder
sys.meta_path.append(NotebookFinder())
import tensorflow as tf
import numpy as np
if __name__ == "__main__":
    sys.path.append("../../")

from notebooks_src.configs import configs
from notebooks_src.postprocessing.utils import get_int_tensor_shape, sort_some_lists_of_tensors

from notebooks_src.box_processing.match import match_boxes
from notebooks_src.box_processing.encode import encode
from notebooks_src.box_processing.unpack import unpack_net_output, split_box_class

from notebooks_src.box_processing.decode import decode
#from notebooks_src.postprocessing.ssd import detected_bboxes
#from notebooks_src.postprocessing.bboxes import bboxes_matching_batch
from notebooks_src.postprocessing.utils import reshape_list
from notebooks_src.metrics.utils import precision_recall, average_precision_voc07, average_precision_voc12, EpochMetrics
from notebooks_src.configs import configs
from notebooks_src.box_processing.tf_box_util import zero_out_negatives, make_box_coords_relative, make_actual_gt_box_mask, ious_with_reference_boxes

from notebooks_src.metrics.mAP import postprocess_boxes_for_map_or_plotting, process_ground_truth_for_map




importing Jupyter notebook from ../../notebooks_src/postprocessing/utils.ipynb
importing Jupyter notebook from ../../notebooks_src/box_processing/match.ipynb
importing Jupyter notebook from ../../notebooks_src/box_processing/tf_box_util.ipynb
importing Jupyter notebook from ../../notebooks_src/box_processing/make_anchors_orig.ipynb
importing Jupyter notebook from ../../notebooks_src/box_processing/encode.ipynb
importing Jupyter notebook from ../../notebooks_src/box_processing/unpack.ipynb
importing Jupyter notebook from ../../notebooks_src/box_processing/decode.ipynb
importing Jupyter notebook from ../../notebooks_src/metrics/utils.ipynb
importing Jupyter notebook from ../../notebooks_src/tf_extended/math.ipynb
importing Jupyter notebook from ../../notebooks_src/metrics/mAP.ipynb


In [14]:
def calc_tp_fp(cbboxes, gt_bboxes, cscores):
    tp_fp_dict = {k:{"tps":[], "fps":[], "scores":[]} for k in cbboxes.keys()}
    
    for c in cbboxes.keys():
        print c
        bboxes = cbboxes[c]
        scores = cscores[c]
        gt_bboxes_of_class_c = gt_bboxes[c]
        print gt_bboxes_of_class_c
        tps, fps = calc_tp_fp_for_class(bboxes, gt_bboxes_of_class_c)
        tp_fp_dict[c]["tps"] = tps
        tp_fp_dict[c]["fps"] = fps
        tp_fp_dict[c]["scores"] = cscores[c]
    return tp_fp_dict
        

def calc_tp_fp_for_class(bboxes, gt_bboxes):
    all_tps = []
    all_fps = []
    for ex in range(bboxes.get_shape()[0].value):

        tps, fps= calc_tf_fp_for_one_example_one_class(bboxes[ex],
                                                      gt_bboxes[ex])
        
        all_tps.append(tps)
        all_fps.append(fps)
    all_tps = tf.concat(all_tps,axis=0)
    all_fps = tf.concat(all_fps,axis=0)
    
    return all_tps, all_fps
        
    

In [57]:
def test_calc(cbboxes, gt_bboxes):
    c= 0
    bboxes = cbboxes[c][1]
    gt_bboxes_c = gt_bboxes[c][1]
    return calc_tf_fp_for_one_example_one_class(bboxes_1_example=bboxes, gt_bboxes_1_example=gt_bboxes_c)
    
def calc_tf_fp_for_one_example_one_class(bboxes_1_example, gt_bboxes_1_example):
    
    
    gt_bboxes = gt_bboxes_1_example
    bboxes = bboxes_1_example
    
    # right now boxes not of this class are zeroed out, so some potentially nonconsecutive indices into gt_bboxes
    # are nonzero, so we want to pull those out
    box_mask = make_actual_gt_box_mask(gt_bboxes)
    nonzero_indices = tf.where(box_mask)
    gt_bboxes = tf.squeeze(tf.gather(gt_bboxes, nonzero_indices),axis=1)
    
    
    
    ################
    any_gt_boxes = tf.reduce_any(box_mask)


    calc_idx_map = make_map_func(gt_bboxes)
    max_ind = tf.map_fn(elems=bboxes, fn=calc_idx_map)
    arange = tf.range(0, configs["keep_top_k"], dtype=tf.int64)
    
    # for every ground truth box, pick the first predicted box that had the best overlap with it
    # set that arange to -1 for that predicted box index
    for box_ind in range(configs["num_max_boxes"]):
        
        box_ind = tf.cast(box_ind, dtype=max_ind.dtype)
        gt_mask = tf.equal(max_ind, box_ind )
        any_matches = tf.reduce_any(gt_mask)
        
        # get the min of this array of 0's and 1's where 0's are matches (essentially gets the first match)
        best_box_ind = tf.argmin(tf.cast(tf.logical_not(gt_mask),dtype=tf.int32))
        
        # if no matches then set to an index we won't see 
        best_box_ind = tf.where(any_matches, best_box_ind, configs["keep_top_k"] + 1 )
        
        arange = tf.where(tf.equal(arange, best_box_ind), -1*tf.ones_like(arange), arange)
    
    # all elements of arange that are -1 correspond to a true positive
    tps = tf.where(arange < 0, tf.ones_like(arange), tf.zeros_like(arange) )
    tps = tf.cast(tps,dtype=bool)
    fps = tf.logical_not(tps)
                                                
        
    return tps, fps


    

def make_map_func(gt_bboxes):
    def calc_gt_bbox_idx_one_box(bbox):
        
        #return the gt box index for the box with best overlap or if there
        # are no boxes with overlap > 0.5 then return -1
        ious = ious_with_reference_boxes(bbox, gt_bboxes)
        max_iou = tf.reduce_max(ious)
        argmax_iou = tf.arg_max(ious,dimension=0)
        configs["matching_threshold"] = 0.00001
        gt_thresh = tf.where(max_iou >= configs["matching_threshold"], tf.constant(True), 
                                             tf.constant(False) )
        index = tf.where(gt_thresh, tf.cast(argmax_iou,dtype=bbox.dtype), tf.constant(-1,dtype=bbox.dtype))
        return index
    return calc_gt_bbox_idx_one_box
    
    
    

# def calc_tp_fp_for_one_box(bbox, gt_bboxes, already_match):
#     tp_or_not = tf.constant(True,dtype=tf.bool)
#     ious = ious_with_reference_boxes(bbox, gt_bboxes)
#     max_iou = tf.maximum(ious)
#     argmax_iou = tf.arg_max(ious)
#     gt_thresh = tf.where(max_iou > 0.5, tf.constant(True), 
#                                          tf.constant(False) )
#     not_already_match = tf.where(already_match[argmax_iou],
#                                  tf.constant(False),
#                                  tf.constant(True))
#     tp_or_not = np.logical_and(gt_thresh, not_already_match)
#     already_match[argmax_iou] = tp_or_not
#     return tp_or_not, already_match

In [58]:
if __name__ == "__main__":
    with tf.Session() as sess:
        bs = configs['batch_size']
        import h5py
        y_true = tf.placeholder(tf.float32,shape=(bs,15,5))
        box = h5py.File(configs["tr_data_file"])["boxes"][323:328]
        shapes = [
                    (bs, 6, 9, 54),
                    (bs, 3, 5, 36),
                    (bs, 96, 144, 36),
                    (bs, 24, 36, 54),
                    (bs, 12, 18, 54),
                    (bs, 48, 72, 54),
                    (bs, 1, 1, 36)]

        y_true = box #.astype("float32")
        y_preds = [tf.random_normal(mean=0.0,stddev=3.,shape=shape) for shape in shapes]
        cgt_bboxes = process_ground_truth_for_map(y_true)
        cbboxes, cscores = postprocess_boxes_for_map_or_plotting(y_preds)
        
        tpfp_dict = sess.run(test_calc(cbboxes, cgt_bboxes))
        

InvalidArgumentError: Reduction axis 0 is empty in shape [0]
	 [[Node: map_43/while/ArgMax = ArgMax[T=DT_FLOAT, Tidx=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"](map_43/while/div, map_43/while/ArgMax/dimension)]]

Caused by op u'map_43/while/ArgMax', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python2.7/dist-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/usr/local/l

In [ ]:
tpfp_dict[0]

In [35]:
tpfp_dict[1]

{0: array([[[ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.]],
 
        [[ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.]],
 
        [[ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],

In [208]:
scores

array([ 0.99998796,  0.99998736,  0.99998641,  0.99998105,  0.99997997,
        0.99997687,  0.99997485,  0.99997079,  0.99995315,  0.99994707,
        0.99994266,  0.9999398 ,  0.99993932,  0.99992776,  0.99992609,
        0.99992371,  0.99991071,  0.99991047,  0.9999088 ,  0.99990654,
        0.99989367,  0.99989164,  0.99988198,  0.99988163,  0.99988067,
        0.99987268,  0.99986935,  0.99986494,  0.9998647 ,  0.99985003,
        0.99983776,  0.9998374 ,  0.9998306 ,  0.99982709,  0.9998228 ,
        0.99982172,  0.99981934,  0.9998166 ,  0.99980456,  0.99980396,
        0.99979991,  0.99979633,  0.99978989,  0.99978822,  0.99978334,
        0.99976677,  0.99976426,  0.99976379,  0.99976295,  0.99975401,
        0.99974674,  0.99972767,  0.99971157,  0.99971014,  0.99970537,
        0.999681  ,  0.9996779 ,  0.99966145,  0.99965954,  0.99962747,
        0.99960691,  0.99960619,  0.99960369,  0.99960238,  0.99960059,
        0.9995926 ,  0.99959224,  0.99958223,  0.99957091,  0.99

In [173]:
csh = c.get_shape()

In [176]:
csh.assert_has_rank(2)

In [177]:
d = tf.constant([])

In [178]:
dsh = d.get_shape()

In [184]:
try:
    f = d[0]
except:
    f = 9

In [185]:
f

9

In [195]:
tf.segment_min?